In [8]:
import fitz
import re 
from langchain_core.documents import Document


In [9]:
def extract_text_from_pdf(pdf_path : str) : 
    try : 
        with fitz.open(pdf_path) as doc : 
            full_text = "".join(page.get_text() for page in doc)
        return full_text
    except Exception as e : 
        print(e)
        return ""
    
pdf_file_path = "/workspace/경찰공무원 등의 개인정보 처리에 관한 규정(대통령령)(제35039호)(20241203).pdf"
# extracted_text = extract_text_from_pdf(pdf_file_path)
# print(extracted_text)


In [ ]:
def chunk_korean_law_by_article(text, source) : 
    # 예: "제15조(개인정보의 수집ㆍ이용)", "제28조의2(가명정보의 처리 등)"
    pattern = r"(제\d+조(?:의\d+)?\s*\(.+?\))" # /를 )로 수정하여 괄호 짝을 맞춤

    split_text = re.split(pattern, text)

    documents = []
    for i in range(1, len(split_text), 2) : 
        article_title = split_text[i]
        article_content = split_text[i+1].strip()

        full_content = f"{article_title}\n{article_content}"

        doc = Document(
            page_content=full_content,
            metadata={
                "source":source,
                "article":article_title.split('(')[0].strip()
            }
        )
        documents.append(doc)
    return documents

In [26]:
extracted_text = extract_text_from_pdf(pdf_file_path)

if extracted_text:
    # 2. 텍스트를 '조' 단위로 분할하여 Document 객체 리스트 생성 (이번 단계)
    law_documents = chunk_korean_law_by_article(extracted_text, pdf_file_path)
    
    print(f"\n✅ 총 {len(law_documents)}개의 '조(Article)'로 문서를 분할했습니다.")
    
    if law_documents:
        print("\n--- [첫 번째 분할 결과 샘플] ---")
        print(law_documents[0])
        
        print("\n--- [중간 분할 결과 샘플 (2번째)] ---")
        print(law_documents[1]) # 제15조는 보통 15번째에 위치
else:
    print("\n❌ 텍스트가 추출되지 않아 분할을 진행할 수 없습니다.")


✅ 총 5개의 '조(Article)'로 문서를 분할했습니다.

--- [첫 번째 분할 결과 샘플] ---
page_content='제1조(목적)
이 영은 경찰공무원 및 경찰청ㆍ해양경찰청 소속 직원 등이 그 직무를 수행하기 위하여 필요한 경우 「개인
정보 보호법」 제23조, 제24조 및 제24조의2에 따른 개인정보를 처리할 수 있는 근거를 규정함을 목적으로 한다.' metadata={'source': '/workspace/경찰공무원 등의 개인정보 처리에 관한 규정(대통령령)(제35039호)(20241203).pdf', 'article': '제1조'}

--- [중간 분할 결과 샘플 (2번째)] ---
page_content='제2조(경찰공무원 등의 민감정보 등의 처리)
경찰공무원 및 경찰청ㆍ해양경찰청 소속 직원은 다음 각 호의 업무를 수행
하기 위하여 불가피한 경우 「개인정보 보호법」 제23조에 따른 민감정보, 같은 법 제24조에 따른 고유식별정보 및 같
은 법 제24조의2에 따른 주민등록번호가 포함된 자료를 처리할 수 있다.
1. 「경찰관 직무집행법」 제2조에 따른 경찰관의 직무
2. 「형사소송법」에 따른 범인ㆍ범죄사실 및 그 증거에 대한 수사
3. 「디엔에이신원확인정보의 이용 및 보호에 관한 법률」에 따른 디엔에이감식시료의 채취, 디엔에이감식시료채취
영장의 신청, 디엔에이신원확인정보의 수록ㆍ관리, 검색ㆍ회보, 삭제, 디엔에이감식시료의 폐기 등 디엔에이신원
확인정보의 수집ㆍ이용 및 보호에 관한 업무
4. 주민등록 관계 법령에 따른 주민등록증발급신청서의 관리, 검색ㆍ대조 등에 관한 업무
5. 경찰공무원 및 경찰청ㆍ해양경찰청 소속 직원의 인사기록 작성ㆍ유지ㆍ변경ㆍ보관, 임용에 필요한 자격 및 요건
의 확인, 그 밖에 시험과 임용 등 인사사무의 처리를 위한 업무
6. 경찰청 또는 해양경찰청 내부 정보시스템 등 정보통신망 운영ㆍ관리에 관한 업무
7. 제1호부터 제6호까지의 업무를 수행하기 위하여 부수

In [ ]:
# jsonl과 pdf를 각각 추출하고 이를 기반으로 하나의 거대 RAG를 생성한다.

In [2]:
import fitz  # PyMuPDF
import re
import json
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 1. 처리할 파일 목록 정의 ---
pdf_file_paths = [
    "/workspace/개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률(법률)(제20369호)(20241017).pdf",
    "/workspace/개인정보 보호법(법률)(제19234호)(20250313) (1).pdf",
    "/workspace/거버넌스.pdf",
    "/workspace/경찰공무원 등의 개인정보 처리에 관한 규정(대통령령)(제35039호)(20241203).pdf",
    "/workspace/금융보안연구원.pdf",
    "/workspace/금융소비자 보호에 관한 법률(법률)(제20305호)(20240814).pdf",
    "/workspace/금융실명거래 및 비밀보장에 관한 법률 시행규칙(총리령)(제01406호)(20170726).pdf",
    "/workspace/랜섬웨어.pdf",
    "/workspace/마이데이터.pdf",
    "/workspace/메타버스.pdf",
    "/workspace/법원 개인정보 보호에 관한 규칙(대법원규칙)(제03109호)(20240315).pdf",
    "/workspace/아웃소싱.pdf",
    "/workspace/정보_보안.pdf",
    "/workspace/클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률(법률)(제20732호)(20250131).pdf",
]
jsonl_file_paths = [
    "/workspace/2025-AI-Challeng-finance/cybersecurity_data_regex_cleaned.jsonl",
    "/workspace/2025-AI-Challeng-finance/qa_data.jsonl"
]

# --- 2. 각 파일 유형별 처리 함수 정의 ---

def extract_text_from_pdf(pdf_path: str) -> str:
    """PDF 파일에서 텍스트를 추출합니다."""
    try:
        with fitz.open(pdf_path) as doc:
            return "".join(page.get_text() for page in doc)
    except Exception as e:
        print(f"'{pdf_path}' 처리 중 오류: {e}")
        return ""

def chunk_korean_law_by_article(text: str, source: str) -> list[Document]:
    """법률 텍스트를 '조' 단위로 분할하여 Document 객체 리스트를 생성합니다."""
    # ... (기존 코드와 동일)
    pattern = r"(제\d+조(?:의\d+)?\s*\(.+?\))"
    split_text = re.split(pattern, text)
    documents = []
    for i in range(1, len(split_text), 2):
        article_title = split_text[i]
        article_content = split_text[i+1].strip()
        if not article_content: continue
        doc = Document(
            page_content=f"{article_title}\n{article_content}",
            metadata={"source": source, "article": article_title.split('(')[0].strip(), "data_type": "legal_pdf"}
        )
        documents.append(doc)
    return documents

# ##################################################################
# ############# ✨ 일반 PDF 처리를 위한 새 모듈(함수) ✨ #############
# ##################################################################
def chunk_general_pdf(text: str, source: str) -> list[Document]:
    """일반 텍스트를 의미 기반의 청크로 분할하여 Document 객체 리스트를 생성합니다."""
    if not text:
        return []
    
    # LangChain의 RecursiveCharacterTextSplitter를 사용하여 텍스트를 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,      # 각 청크의 최대 크기
        chunk_overlap=100,    # 청크 간 중복되는 글자 수
        separators=["\n\n", "\n", " ", ""] # 문단 > 문장 > 단어 순으로 분할 시도
    )
    
    # 텍스트를 직접 분할하여 문자열 리스트를 얻음
    chunks = text_splitter.split_text(text)
    
    # 각 청크 문자열을 Document 객체로 변환
    documents = []
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk,
            metadata={"source": source, "chunk_num": i + 1, "data_type": "general_pdf"}
        )
        documents.append(doc)
        
    return documents

# --- 3. 메인 실행: 모든 파일 처리 및 통합 ---

all_documents = []

print("--- PDF 파일 처리 시작 ---")
for pdf_path in pdf_file_paths:
    print(f"'{pdf_path}' 파일 처리 중...")
    extracted_text = extract_text_from_pdf(pdf_path)
    if not extracted_text:
        print(f"-> 텍스트가 없어 건너뜁니다.")
        continue

    # 파일 이름에 '법', '령', '규칙'이 포함되어 있는지 확인하여 분기 처리
    if any(keyword in pdf_path for keyword in ["법", "령", "규칙"]):
        documents = chunk_korean_law_by_article(extracted_text, pdf_path)
        all_documents.extend(documents)
        print(f"-> [법률] {len(documents)}개의 '조(Article)'를 추가했습니다.")
    else:
        documents = chunk_general_pdf(extracted_text, pdf_path)
        all_documents.extend(documents)
        print(f"-> [일반] {len(documents)}개의 '청크'를 추가했습니다.")

# --- JSONL 파일 처리 ---
print("\n--- JSONL 파일 처리 시작 ---")
# ... (기존 코드와 동일) ...
text_splitter_jsonl = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
for jsonl_path in jsonl_file_paths:
    print(f"'{jsonl_path}' 파일 처리 중...")
    try:
        loader = JSONLoader(
            file_path=jsonl_path,
            jq_schema='"질문: " + .question + "\\n답변 : " + .answer', 
            json_lines=True
        )
        documents_from_jsonl = loader.load()
        split_docs = text_splitter_jsonl.split_documents(documents_from_jsonl)
        for doc in split_docs:
            doc.metadata["data_type"] = "qna_jsonl"
        all_documents.extend(split_docs)
        print(f"-> {len(split_docs)}개의 'Q&A' 청크를 추가했습니다.")
    except Exception as e:
        print(f"'{jsonl_path}' 처리 중 오류: {e}")

# --- 최종 결과 확인 ---
print("\n--- ✅ 최종 통합 완료 ---")
print(f"모든 파일로부터 총 {len(all_documents)}개의 Document를 생성했습니다.")

if all_documents:
    print("\n--- [통합된 데이터 샘플 확인] ---")
    legal_pdf_sample = next((doc for doc in all_documents if doc.metadata.get("data_type") == "legal_pdf"), None)
    if legal_pdf_sample:
        print("📄 법률 PDF에서 온 데이터 샘플:")
        print(legal_pdf_sample)
    
    general_pdf_sample = next((doc for doc in all_documents if doc.metadata.get("data_type") == "general_pdf"), None)
    if general_pdf_sample:
        print("\n📑 일반 PDF에서 온 데이터 샘플:")
        print(general_pdf_sample)
    
    jsonl_sample = next((doc for doc in all_documents if doc.metadata.get("data_type") == "qna_jsonl"), None)
    if jsonl_sample:
        print("\n📝 JSONL에서 온 데이터 샘플:")
        print(jsonl_sample)

--- PDF 파일 처리 시작 ---
'/workspace/개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률(법률)(제20369호)(20241017).pdf' 파일 처리 중...
-> [일반] 30개의 '청크'를 추가했습니다.
'/workspace/개인정보 보호법(법률)(제19234호)(20250313) (1).pdf' 파일 처리 중...
-> [일반] 96개의 '청크'를 추가했습니다.
'/workspace/거버넌스.pdf' 파일 처리 중...
-> [일반] 3개의 '청크'를 추가했습니다.
'/workspace/경찰공무원 등의 개인정보 처리에 관한 규정(대통령령)(제35039호)(20241203).pdf' 파일 처리 중...
-> [일반] 2개의 '청크'를 추가했습니다.
'/workspace/금융보안연구원.pdf' 파일 처리 중...
-> [일반] 2개의 '청크'를 추가했습니다.
'/workspace/금융소비자 보호에 관한 법률(법률)(제20305호)(20240814).pdf' 파일 처리 중...
-> [일반] 53개의 '청크'를 추가했습니다.
'/workspace/금융실명거래 및 비밀보장에 관한 법률 시행규칙(총리령)(제01406호)(20170726).pdf' 파일 처리 중...
-> [일반] 1개의 '청크'를 추가했습니다.
'/workspace/랜섬웨어.pdf' 파일 처리 중...
-> [일반] 4개의 '청크'를 추가했습니다.
'/workspace/마이데이터.pdf' 파일 처리 중...
-> [일반] 3개의 '청크'를 추가했습니다.
'/workspace/메타버스.pdf' 파일 처리 중...
-> [일반] 12개

In [3]:
## huggingface 객관식 데이터를 question,answer형태로변환

In [ ]:
# # generate english rag

# import json 

# file_path = "/workspace/merged_questions.json"
# with open(file_path, "r", encoding="utf-8") as f:
#     dataset = json.load(f)

# print(len(dataset["questions"]))


# document_len = len(dataset["questions"])
# real_data = dataset["questions"]

# all_docs = []

# for i in range(document_len):

#     question = real_data[i]["question"]
#     # .get() is used to safely access the answer, avoiding errors if the key is missing
#     answer = real_data[i]["answers"].get(real_data[i]["solution"])

#     docs = {
#         "question" : question,
#         "answer" : answer
#     }

#     all_docs.append(docs)

# print(all_docs[10])

# # --- ✨ 여기에 JSONL 저장 코드를 추가합니다 ✨ ---

# # 저장할 파일 이름 정의
# output_filename = "qa_data.jsonl"

# # all_docs 리스트를 jsonl 파일로 저장
# with open(output_filename, 'w', encoding='utf-8') as f:
#     for doc in all_docs:
#         # 각 딕셔너리를 JSON 문자열로 변환하여 파일에 한 줄씩 씁니다.
#         # ensure_ascii=False는 비-ASCII 문자(예: 한글)가 깨지지 않도록 합니다.
#         f.write(json.dumps(doc, ensure_ascii=False) + '\n')

# print(f"\n✅ 데이터가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")


12760
{'question': 'Which of the following authentication factors involves something you know?', 'answer': 'Username'}

✅ 데이터가 'qa_data.jsonl' 파일에 성공적으로 저장되었습니다.


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
import torch 

# --- 4. RAG 구성: 임베딩 및 벡터 스토어 생성 ---

# 사용할 임베딩 모델 ID
embedding_model_id = "BAAI/bge-m3"

print(f"\n--- [RAG 구성 시작] ---")
print(f"'{embedding_model_id}' 임베딩 모델을 로드합니다...")

# HuggingFaceEmbeddings 객체 생성
# model_kwargs: GPU 사용 설정 (사용 가능할 경우)
# encode_kwargs: 임베딩 정규화 설정 (성능 향상에 도움)
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ 임베딩 모델 로드 완료.")

print(f"\n총 {len(all_documents)}개의 Document를 벡터로 변환하여 FAISS DB를 구축합니다...")

# FAISS.from_documents를 사용하여 벡터 DB 생성
# 이 과정에서 tqdm을 직접 사용하기는 어렵지만, 내부적으로 모든 문서를 처리합니다.
vectorstore = FAISS.from_documents(all_documents, embeddings)

print("✅ FAISS 벡터 DB 구축 완료.")
db_save_path = "./faiss_db_kor_eng"
vectorstore.save_local(db_save_path)
# 검색기(Retriever) 생성
# search_kwargs={'k': 3}: 검색 시 가장 유사한 3개의 문서를 가져오도록 설정
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

print("✅ Retriever 생성이 완료되었습니다.")


# --- 5. RAG 검색 테스트 (선택 사항) ---

if retriever:
    print("\n--- [RAG 검색 테스트] ---")
    test_query = "랜섬웨어 공격을 방지하는 방법은 무엇인가요?"
    
    # retriever.invoke()를 사용하여 테스트 쿼리와 관련된 문서 검색
    retrieved_docs = retriever.invoke(test_query)
    
    print(f"❓ 테스트 질문: \"{test_query}\"")
    print(f"\n🔍 검색된 관련 문서 ({len(retrieved_docs)}개):")
    for i, doc in enumerate(retrieved_docs):
        print(f"\n--- [문서 {i+1}] ---")
        # page_content의 앞부분 150자만 출력
        print(doc.page_content[:150] + "...")
        print(f"(출처: {doc.metadata.get('source', 'N/A')})")


--- [RAG 구성 시작] ---
'BAAI/bge-m3' 임베딩 모델을 로드합니다...


✅ 임베딩 모델 로드 완료.

총 15829개의 Document를 벡터로 변환하여 FAISS DB를 구축합니다...
✅ FAISS 벡터 DB 구축 완료.
✅ Retriever 생성이 완료되었습니다.

--- [RAG 검색 테스트] ---
❓ 테스트 질문: "랜섬웨어 공격을 방지하는 방법은 무엇인가요?"

🔍 검색된 관련 문서 (3개):

--- [문서 1] ---
2. 네트워크 접근제어 : 비인가자의 통신망 접속을 적절하게 조절하는 기술. 특정 보안기능을 적용한 후
정보 보안의 방법
에 접속할 수 있게 하므로 네트워크 장애를 발생시키는 빈도를 줄임.[5]
3. 악성코드차단: 악성코드 종류는 다양하다. 컴퓨터 바이러스, 웜, 스파...
(출처: /workspace/정보_보안.pdf)

--- [문서 2] ---
WannaCry 랜섬웨어에 감염된 모습
랜섬웨어
랜섬웨어(영어: ransomware)는 몸값을 지불할 때까지 피해자
의 개인 데이터를 암호화하는 악성 소프트웨어의 한 유형이
다. 컴퓨터 시스템을 감염시켜 접근을 제한하고 복호화 키를
조건으로 일종의 몸값을 요구한다. 컴...
(출처: /workspace/랜섬웨어.pdf)

--- [문서 3] ---
질문: Which of the following is a common practice to prevent/troubleshoot ransomware attacks?
답변 : Having up-to-date backup copies...
(출처: /workspace/2025-AI-Challeng-finance/qa_data.jsonl)


: 